In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Traemos los dataframes

In [2]:
itemsxgames = pd.read_parquet('itemsxgames.parquet')
reviewsxgames = pd.read_parquet('reviewsxgames.parquet')
steamxgames = pd.read_parquet('steamxgamesml.parquet')

Puliendo algunos detalles antes de hacer las funciones

In [3]:
funcion_4 = pd.read_parquet(r"data\user_reviews.parquet")

In [4]:
itemsxgames = itemsxgames.query('playtime_forever != 0')

# Imprime el DataFrame resultante
itemsxgames

,genres,year_release,playtime_forever,user_id
0,Action,2000,6,76561197970982479
1,Action,2000,93,doctr
2,Action,2000,108,corrupted_soul
3,Action,2000,328,WeiEDKrSat
4,Action,2000,580,Fr0stedLine
...,...,...,...,...
4194864,Indie,2016,164,76561198107283457
4194865,Simulation,2016,164,76561198107283457
4194866,Action,2015,1,76561198146468235
4194867,Indie,2015,1,76561198146468235


In [5]:
merged_df = pd.merge(funcion_4, steamxgames, on='item_id', how='inner')
merged_df

,user_id,item_id,recommend,year_posted,sentiment_analysis,title,developer,description
0,76561197970982479,1250,1,2011,2,Killing Floor,Tripwire Interactive,"Action, FPS, Zombies, Co-op, Survival, Action,..."
1,death-hunter,1250,1,2015,2,Killing Floor,Tripwire Interactive,"Action, FPS, Zombies, Co-op, Survival, Action,..."
2,DJKamBer,1250,1,2013,0,Killing Floor,Tripwire Interactive,"Action, FPS, Zombies, Co-op, Survival, Action,..."
3,diego9031,1250,1,2015,1,Killing Floor,Tripwire Interactive,"Action, FPS, Zombies, Co-op, Survival, Action,..."
4,76561198081962345,1250,1,2014,1,Killing Floor,Tripwire Interactive,"Action, FPS, Zombies, Co-op, Survival, Action,..."
...,...,...,...,...,...,...,...,...
49286,llDracuwulf,307130,1,2015,2,Asteria,Legend Studio,"Action, Adventure, Indie, Indie, Adventure, Ac..."
49287,ChrisCoroner,209120,1,2013,2,Street Fighter X Tekken,"Capcom U.S.A., Inc.","Action, Fighting, Action, Arcade, 2D Fighter, ..."
49288,MeloncraftLP,220090,1,2013,1,The Journey Down: Chapter One,SkyGoblin,"Adventure, Indie, Adventure, Point & Click, In..."
49289,MeloncraftLP,262850,1,2013,1,The Journey Down: Chapter Two,SkyGoblin,"Adventure, Indie, Adventure, Indie, Point & Cl..."


### Funciones

In [6]:
def PlayTimeGenre(genero: str):
    '''
    Debe devolver año con mas horas jugadas para dicho género.
    Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}
    
    '''
    # Filtramos el dataframe 'items_games' respecto al parámetro genero
    df = itemsxgames[itemsxgames['genres']== genero]
    
    # Agrupamos el dataframe anterior por año de lanzamiento, suma de minutos de juego y ordenamos en forma descendente
    agrupado = df.groupby('year_release')['playtime_forever'].sum().sort_values(ascending=False)

    # El valor máximo tendrá índice [0]
    anio = agrupado.index[0]
    
    return {f"Año de lanzamiento con más horas jugadas para el género {genero}": int(anio)}

In [7]:
PlayTimeGenre('Action')

{'Año de lanzamiento con más horas jugadas para el género Action': 2013}

In [8]:
def UserForGenre(genero: str):
    '''
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}
    '''
    # Filtramos el dataframe 'itemsxgames' respecto al parámetro genero
    df = itemsxgames[itemsxgames['genres'] == genero]

    # Agrupamos el dataframe anterior por usuario, suma de horas de juego y ordenamos en forma descendente
    agrupado = df.groupby('user_id')['playtime_forever'].sum().sort_values(ascending=False)

    # El valor máximo tendrá índice [0]
    user = agrupado.index[0]

    # Tomamos las filas del dataframe util que contengan su respectivo usuario (user)
    df_genero_user = df[df['user_id'] == user]

    # Agrupamos respecto a los años y suma de horas de juego
    horas_jugadas = round(df_genero_user.groupby('year_release')['playtime_forever'].sum() / 60, 3)

    # Guardamos la serie 'horas_jugadas' en una lista
    lista_horas_jugadas = [{'Año': int(anio), 'Horas': horas} for anio, horas in horas_jugadas.items()]

    return {f"Usuario con más horas jugadas para género {genero}": user, "Horas jugadas": lista_horas_jugadas}


In [9]:
UserForGenre('Simulation')

{'Usuario con más horas jugadas para género Simulation': 'phrostb',
 'Horas jugadas': [{'Año': 1994, 'Horas': 5.233},
  {'Año': 1996, 'Horas': 3.133},
  {'Año': 1998, 'Horas': 6.1},
  {'Año': 2000, 'Horas': 8.7},
  {'Año': 2001, 'Horas': 2.8},
  {'Año': 2002, 'Horas': 1.183},
  {'Año': 2003, 'Horas': 2.333},
  {'Año': 2004, 'Horas': 4.067},
  {'Año': 2006, 'Horas': 5.167},
  {'Año': 2007, 'Horas': 10.367},
  {'Año': 2008, 'Horas': 0.2},
  {'Año': 2009, 'Horas': 16.533},
  {'Año': 2010, 'Horas': 11.233},
  {'Año': 2011, 'Horas': 21.883},
  {'Año': 2012, 'Horas': 35.333},
  {'Año': 2013, 'Horas': 86.467},
  {'Año': 2014, 'Horas': 186.467},
  {'Año': 2015, 'Horas': 432.45},
  {'Año': 2016, 'Horas': 245.483},
  {'Año': 2017, 'Horas': 10.8},
  {'Año': 2018, 'Horas': 4.15}]}

In [10]:
def UserRecommend(anio: int):
    '''
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
    Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

    '''
    # Si el año de lanzamiento(year_release) no coincide con alguno de los años en los que se hace una reseña(year_posted), se retorna un mensaje de erro
    if anio not in reviewsxgames['year_posted'].unique():
        return f"Año fuera de rango, ingrese un año válido"
    
    else:
        # Filtramos el dataframe con las filas cuyo año de posteo(year_posted) es mayor o igual al año de publicación(year_release)
        df = reviewsxgames[reviewsxgames['year_posted']>=reviewsxgames['year_release']]
        
        # Filtramos el dataframe 'df' para el año parámetro y la columna sentiment_analysis sea positivo(2) o neutro(1)
        df_anio_dado = df[(df['year_posted']==anio) & (df['sentiment_analysis'].isin([1,2]))]

        # Agrupamos el dataframe 'df_anio_dado' por título del juego ('title'), sumamos las recomendaciones('recommend') para tener los juegos más recomendados y ordenamos de forma descendente
        top = df_anio_dado.groupby('title')['recommend'].sum().sort_values(ascending=False)

        # Construimos el top3
        top3 = [{"Puesto 1": top.index[0]}, {"Puesto 2": top.index[1]}, {"Puesto 3": top.index[2]}]

    return top3

In [11]:
UserRecommend(2012)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Terraria'},
 {'Puesto 3': 'Realm of the Mad God'}]

In [12]:
def UsersWorstDeveloper(año: int):
    '''
    Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
    Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
    '''
    # Filtra el DataFrame para el año dado
    filtered_df = merged_df[merged_df['year_posted'] == año]

    # Filtra juegos menos recomendados (recommend=0 y sentiment_analysis<2)
    less_recommended = filtered_df[(filtered_df['recommend'] == 0) & (filtered_df['sentiment_analysis'] < 2)]

    # Agrupa por desarrolladora y cuenta la cantidad de juegos menos recomendados
    developer_counts = less_recommended.groupby('developer').size().reset_index(name='count')

    # Ordena en orden ascendente por la cantidad de juegos menos recomendados
    sorted_developers = developer_counts.sort_values(by='count')

    # Toma las 3 desarrolladoras con menos juegos recomendados
    top_3 = sorted_developers.head(3)

    # Crea el resultado en el formato solicitado
    result = [{"Puesto {}: {}".format(i+1, row['developer'])} for i, (index, row) in enumerate(top_3.iterrows())]

    return result

In [13]:
UsersWorstDeveloper(2015)

[{'Puesto 1: 10th Art Studio,Adventure Productions'},
 {'Puesto 2: Orange_Juice'},
 {'Puesto 3: Oovee® Game Studios'}]

In [14]:
def sentiment_analysis(desarrolladora: str):
    '''
    Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
    Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}
    '''
    # Filtra el DataFrame para la desarrolladora dada
    filtered_df = merged_df[merged_df['developer'] == desarrolladora]

    # Cuenta la cantidad de registros por análisis de sentimiento
    sentiment_counts = filtered_df['sentiment_analysis'].value_counts().sort_index()

    # Mapea los valores numéricos de sentiment_analysis a etiquetas
    sentiment_labels = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    sentiment_counts.rename(index=sentiment_labels, inplace=True)

    # Crea el resultado en el formato solicitado
    result = {desarrolladora: sentiment_counts.to_dict()}

    return result


In [15]:
sentiment_analysis('Valve')

{'Valve': {'Negative': 1114, 'Neutral': 2357, 'Positive': 6105}}

**Modelo recomendacion_juego( id de producto )**

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Creamos una instancia de la clase CountVectorizer
vector = CountVectorizer(tokenizer=lambda x: x.split(', '))

# Dividimos cada cadena de descripción en palabras individuales y creamos una matriz de conteo 'matriz_generos' que representa cuántas veces aparece cada género en cada videojuego.
matriz_generos = vector.fit_transform(steamxgames['description'])

def recomendacion_juego(id_producto: int):
    '''
    Se ingresa el id de producto (item_id) y retorna una lista con 5 juegos recomendados similares al ingresado (title).
    '''
    # Si el id ingresado no se encuentra en la columna de id de la tabla 'steam_games', se le pide al usuario que intente con otro id
    if id_producto not in steamxgames['item_id'].values:
        return 'El ID no existe, intente con otro'
    else:
        # Buscamos el índice del id ingresado
        index = steamxgames.index[steamxgames['item_id'] == id_producto][0]

        # De la matriz de conteo, tomamos el array de géneros con índice igual a 'index'
        generos_index = matriz_generos[index]

        # Calculamos la similitud coseno entre los géneros de entrada y los géneros de las demás filas: cosine_similarity(generos_index, matriz_generos)
        # Obtenemos los índices de las mayores similitudes mediante el método argsort() y las similitudes ordenadas de manera descendente
        # Tomamos los índices del 1 al 6 [0, 1:6] ya que el índice 0 es el mismo índice de entrada
        indices_maximos = np.argsort(-cosine_similarity(generos_index, matriz_generos))[0, 1:6]

        # Construimos la lista de recomendaciones
        recomendaciones = []
        for i in indices_maximos:
            recomendaciones.append(steamxgames['title'][i])

        return recomendaciones


c:\Users\ramir\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [17]:
recomendacion_juego(745400)

['Dark Snow',
 'Cyborg Arena',
 'M1: A Death in the Desert',
 'The Moon Night',
 'Swingin Swiggins']